In [1]:
#import packages
import csv
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import math
from joblib import dump, load

In [24]:
#data normalization
def normalize(data):
    data_ = []
    for i in range(len(data)):
        row = []
        for j in range(len(data[i])):
            if j == 2:
                row.append(int(data[i][j])/12)
            elif j == 3:
                val = (float(data[i][j]) +60)/60
                row.append(val)
            elif j == 10:
                row.append(int(round(float(data[i][j])))/200)
            else:
                row.append(float(data[i][j]))
        data_.append(row)
    return data_

In [25]:
#load centroids from step2 clustering results
def getCentroids():
    cs = []
    with open("centroids_2.csv", "r") as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            drow = []
            for col in row:
                drow.append(float(col))
            cs.append(drow)
    #cs = np.array(cs)
    return cs

In [26]:
#每個sample使用features如下，共11個。
#['danceability','energy','key', 'loudness', 'mode', 'speechiness', 'acousticness','instrumentalness',
# 'liveness','valence','tempo']
#classify a feature vector. x must be a 1x11 vector(11 features)
def classify(centroids,x):
    minDis = 0
    belong = 0
    for i in range(len(centroids)):
        val = 0
        for j in range(len(centroids[0])):
            val += math.pow(x[j]-centroids[i][j],2)
        if i == 0:
            minDis = val
        elif val < minDis:
            minDis = val
            belong = i
    return belong

In [27]:
#load pca model from step2
pca = load("pca.joblib")

In [28]:
#get centroids
centroids = getCentroids()

In [29]:
#get cluster amount
clusterAmount = len(centroids)

In [30]:
#load data
data = []
header = []
if True:
    with open("city_result_2.csv", "r") as csvfile:
        rows = csv.reader(csvfile)
        i = 0
        for row in rows:
            if i == 0:
                header = row[2:13]
            else:
                data.append(row[2:13])
            i += 1

In [31]:
data = normalize(data)

In [32]:
#transform data with pca model
data = np.array(data, dtype=float)
data = pca.transform(data)

In [33]:
nums = []
for i in range(clusterAmount):
    nums.append(0)

In [34]:
#classify every sample in data and output nums of each class
for i in range(len(data)):
    nums[classify(centroids, data[i])]+=1

In [35]:
nums

[151, 148, 85, 184, 177, 72, 102, 107, 46, 198, 146, 93, 190, 109, 192]

In [36]:
centroids

[[0.745066160305548, -0.1685868171405762, 0.2779166849635348],
 [-0.325876719706544, -0.0530926084610796, 0.08991826294473895],
 [0.5726314595441865, 0.36372344702296655, -0.3276249632209552],
 [-0.27253546159341757, -0.3695850798990985, 0.09849434394387359],
 [-0.2436423392341106, -0.30731063439768197, 0.3762352727617841],
 [-0.4829232007672995, 0.6602387543624972, -0.19289254226289626],
 [-0.3971108106098521, 0.5615893303109205, 0.2952195054636615],
 [0.6311409532749043, -0.09969099553731361, -0.43665942383560896],
 [0.5691758430888653, 0.7069113579431354, 0.05592324403966571],
 [-0.33518789971180124, -0.35375844482718694, -0.3230185513885366],
 [-0.32198112700191245, 0.14691946164279007, 0.34179099972232385],
 [-0.42719629315696167, 0.3218010367637969, -0.195226409330275],
 [0.6819854580417434, -0.10116138554052824, -0.08465143545578432],
 [0.6732726026692112, 0.21392855952394013, 0.20699228093116817],
 [-0.3801131721884603, -0.054095922978470296, -0.30202086083266577]]

In [38]:
#load data
test = []
#header = []
if True:
    with open("../playlistresult.csv", "r") as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            test.append(row[:11])

In [39]:
len(test)

100

In [40]:
test = normalize(test)

In [41]:
test[:3]

[[0.746,
  0.69,
  0.9166666666666666,
  0.8674,
  1.0,
  0.164,
  0.247,
  0.0,
  0.101,
  0.497,
  0.45],
 [0.514,
  0.73,
  0.08333333333333333,
  0.9011,
  1.0,
  0.0598,
  0.00146,
  9.54e-05,
  0.0897,
  0.334,
  0.855],
 [0.77,
  0.724,
  0.6666666666666666,
  0.9086,
  1.0,
  0.0495,
  0.0167,
  0.0107,
  0.353,
  0.898,
  0.61]]

In [42]:
#transform data with pca model
test = np.array(test, dtype=float)
test = pca.transform(test)

In [43]:
numstest = []
for i in range(clusterAmount):
    numstest.append(0)

In [44]:
#classify every sample in data and output nums of each class
for i in range(len(test)):
    numstest[classify(centroids, test[i])]+=1

In [45]:
numstest

[13, 7, 2, 8, 8, 3, 5, 5, 4, 7, 11, 8, 5, 4, 10]